<a href="https://colab.research.google.com/github/Worachet-Ch/project2022/blob/main/column_of_random_NaN_by_Probability_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error
import time
from sklearn import neighbors
from sklearn.impute import KNNImputer
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import os
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/project' #เข้า drive
data_file_path = os.path.join(path,'dataset2.csv') #ไปที่ไฟล์ confirmed-cases-since-280265.csv
data = pd.read_csv(data_file_path) #อ่านไฟล์ csv

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data

In [ ]:
data

,No.,LabNo,Sex,AgeSYear,Enose,Overall,Body shape age,Kidney,Heart,Liver,...,071_HbA1c,CK,LD-P,AMY7,TP1,DBIL,TBIL,ALB1,GGT,homocysteine
0,1.0,NaN,2,41.0,NaN,NaN,NaN,NaN,NaN,NaN,...,4.8,79.0,231.0,59.0,7.16,0.10,0.30,4.48,11.0,10.88
1,2.0,201807163166.00,1,49.0,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.11
2,3.0,201807163167,1,44.0,NaN,NaN,NaN,NaN,NaN,NaN,...,4.6,85.0,175.0,104.0,7.43,0.15,1.26,4.70,15.0,11.94
3,4.0,201807163168,2,37.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.01
4,5.0,201807163169,2,38.0,NaN,NaN,NaN,NaN,NaN,NaN,...,5.4,57.0,177.0,89.0,7.11,0.02,0.20,3.92,24.0,8.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2269,NaN,V17,2,31.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2270,NaN,V19,2,32.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2271,NaN,V20,1,28.0,NaN,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2272,NaN,V21,1,29.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.dropna(subset=['telomere length (kb) '], inplace = True)
data.drop(['No.', 'LabNo','DHEAs (BRIA)'], axis=1,inplace = True)
data.drop(data.columns[2: 10], axis=1,inplace = True)
data.drop(data.columns[27:37], axis=1,inplace = True)
data.drop('telomere length (kb) ',1,inplace = True)

<ipython-input-25-1a7942661c33>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data.drop('telomere length (kb) ',1,inplace = True)


In [ ]:
data

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
49,2,58.0,47.2,156.0,19.4,112.0,64.0,70.0,NaN,NaN,...,0.6,4.8,180.0,97.0,62.0,98.0,25.0,32.0,40.0,5.8
56,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
85,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
103,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
107,2,86.0,49.0,154.0,20.7,129.0,79.0,83.0,NaN,NaN,...,0.7,5.4,183.0,86.0,39.0,127.0,12.0,22.0,82.0,6.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2267,2,25.0,47.0,165.0,17.3,104.0,63.0,89.0,NaN,NaN,...,0.6,4.8,232.0,36.0,96.0,128.0,18.0,12.0,46.0,NaN
2269,2,31.0,72.2,168.0,25.6,123.0,89.0,69.0,NaN,NaN,...,0.6,6.1,187.0,49.0,51.0,127.0,21.0,24.0,71.0,NaN
2270,2,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.7,5.1,222.0,58.0,79.0,132.0,17.0,10.0,34.0,NaN
2272,1,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,6.8,140.0,92.0,48.0,73.0,31.0,75.0,69.0,NaN


In [ ]:
P = 0
p1 = 0
list_mis = []
for i in data:
  if data[i].isna().sum() == 0 :
    continue
  else:
    ms = data[i].isna().sum()
    P = (ms/1734).round(14)
    p1 = p1 + P
    print(f'ความน่าจะเป็นของ {i} = {P}')
    list_mis.append(P)
print('****************************************')
print(f'ความน่าจะเป็นทั้งหมด = {p1}')

ความน่าจะเป็นของ Weight = 0.00461361014994
ความน่าจะเป็นของ Height = 0.00461361014994
ความน่าจะเป็นของ BMI = 0.00461361014994
ความน่าจะเป็นของ Systolic = 0.00461361014994
ความน่าจะเป็นของ Diastolic = 0.00519031141869
ความน่าจะเป็นของ HeartRate = 0.02941176470588
ความน่าจะเป็นของ SMM = 0.12975778546713
ความน่าจะเป็นของ Fat Mass = 0.12975778546713
ความน่าจะเป็นของ % Body fat = 0.12975778546713
ความน่าจะเป็นของ waist to hip radio = 0.12975778546713
ความน่าจะเป็นของ abdominal circumference = 0.13033448673587
ความน่าจะเป็นของ visceral fat area = 0.12975778546713
ความน่าจะเป็นของ 001_Glucose = 0.00057670126874
ความน่าจะเป็นของ 004_Uric Acid = 0.05363321799308
ความน่าจะเป็นของ 012_ALT = 0.00057670126874
ความน่าจะเป็นของ 013_Alkaline Phos = 0.01672433679354
ความน่าจะเป็นของ 071_HbA1c = 0.09630911188005
****************************************
ความน่าจะเป็นทั้งหมด = 1.0


In [ ]:
data.rename(columns={'Fat Mass':'FatMass','% Body fat':'Bodyfat','waist to hip radio':'waisttohip','abdominal circumference':'abdominal',
                    'visceral fat area':'visceral','004_Uric Acid':'UricAcid',''
                                'HDL-C':'HDL','013_Alkaline Phos':'Alkaline_Phos'},inplace = True)

In [ ]:
data_file_path1 = os.path.join(path,'data_notmissing.csv') #ไปที่ไฟล์ confirmed-cases-since-280265.csv
data1 = pd.read_csv(data_file_path1) #อ่านไฟล์ csv

In [ ]:
data1.drop({'Unnamed: 0','telomere length (kb) '},1,inplace = True)

<ipython-input-30-0e7a2b46e4dd>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data1.drop({'Unnamed: 0','telomere length (kb) '},1,inplace = True)


In [ ]:
data1.rename(columns={'Fat Mass':'FatMass','% Body fat':'Bodyfat','waist to hip radio':'waisttohip','abdominal circumference':'abdominal',
                    'visceral fat area':'visceral','004_Uric Acid':'UricAcid',''
                                'HDL-C':'HDL','013_Alkaline Phos':'Alkaline_Phos'},inplace = True)

In [ ]:
data1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [ ]:
list1 = []
for i in data:
  if data[i].isna().sum() == 0:
    continue
  else: 
    list1.append(i)
list1

['Weight',
 'Height',
 'BMI',
 'Systolic',
 'Diastolic',
 'HeartRate',
 'SMM',
 'FatMass',
 'Bodyfat',
 'waisttohip',
 'abdominal',
 'visceral',
 '001_Glucose',
 'UricAcid',
 '012_ALT',
 'Alkaline_Phos',
 '071_HbA1c']

In [ ]:
list_mis

[0.00461361014994,
 0.00461361014994,
 0.00461361014994,
 0.00461361014994,
 0.00519031141869,
 0.02941176470588,
 0.12975778546713,
 0.12975778546713,
 0.12975778546713,
 0.12975778546713,
 0.13033448673587,
 0.12975778546713,
 0.00057670126874,
 0.05363321799308,
 0.00057670126874,
 0.01672433679354,
 0.09630911188005]

# 10 %

## แทนค่า NaN

In [ ]:
for i in range(1,1001):
  np.random.seed(i)
  globals()[f'random_column_{i}'] = np.random.choice(list1,int((153*17)*(10/100)),p=list_mis)
  globals()[f'random_column_{i}'] = list(globals()[f'random_column_{i}'])


In [ ]:
for i in range(1,1001):
  for j in globals()[f'random_column_{i}']:
    globals()[f'count_{i}'] = []
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('Weight'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('Height'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('BMI'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('Systolic'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('Diastolic'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('HeartRate'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('SMM'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('FatMass'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('Bodyfat'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('waisttohip'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('abdominal'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('visceral'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('001_Glucose'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('UricAcid'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('012_ALT'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('Alkaline_Phos'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('071_HbA1c'))

In [ ]:
list2 = []
list3 = []

for i in range(153):
  m = 1/153
  list2.append(i)
  list3.append(m)

In [ ]:
# random row ไม่ซ้ำกัน
for i in range(1,1001):
  np.random.seed(i)
  globals()[f'dt10_{i}'] = data1.copy()
  for j,k in zip(list1,globals()[f'count_{i}']):
    globals()[f'dt10_{i}'][j].ravel()[np.random.choice(list2,k,p= list3, replace=False)] = np.nan

In [ ]:
dt10_1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,NaN,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,NaN,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,NaN,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,NaN,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,NaN,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,NaN,NaN
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,NaN,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,NaN,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation

### Imputation using Mean

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
t_start = time.time()
mean_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean_{i}'] = mean_imputer.fit_transform(globals()[f'dt10_{i}'].copy())
  globals()[f'MSE_MEAN10_{i}'] = mean_squared_error(data1,globals()[f'imputed_mean_{i}'])
t_end = time.time()
t_MEAN10 = (t_end- t_start)

In [ ]:
mse_MEAN10_total = 0 
for i in range(1,1001):
  mse_MEAN10_total = mse_MEAN10_total + globals()[f'MSE_MEAN10_{i}']
AMSE_mean10 = mse_MEAN10_total/1000
print(f'AMSE_mean = {AMSE_mean10}')

AMSE_mean = 11.462440968099067


### Imputation using the Miss Forest

In [ ]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF = MissForest()
for i in range(1,1001):
  globals()[f'imputed_MF_{i}'] = MF.fit_transform(globals()[f'dt10_{i}'].copy())
  globals()[f'MSE_MF10_{i}'] = mean_squared_error(data1,globals()[f'imputed_MF_{i}'])
t_end = time.time()
t_MF10 = (t_end- t_start)

In [ ]:
mse_MF10_total = 0 
for i in range(1,1001):
  mse_MF10_total = mse_MF10_total + globals()[f'MSE_MF10_{i}']
AMSE_MF10 = mse_MF10_total/1000
print(f'AMSE_MF = {AMSE_MF10}')

AMSE_MF = 2.1657967102677564


### Imputation using the KNN

In [ ]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [ ]:
t_start = time.time()
KNN = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN_{i}'] = KNN.fit_transform(globals()[f'dt10_{i}'].copy())
  globals()[f'MSE_KNN10_{i}'] = mean_squared_error(data1,globals()[f'imputed_KNN_{i}'])
t_end = time.time()
t_KNN10 = (t_end- t_start)

In [ ]:
mse_KNN10_total = 0 
for i in range(1,1001):
  mse_KNN10_total = mse_KNN10_total + globals()[f'MSE_KNN10_{i}']
AMSE_KNN10 = mse_KNN10_total/1000
print(f'AMSE_KNN = {AMSE_KNN10}')

AMSE_KNN = 8.131331247076591


### Imputation using the MICE

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

#### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_LR_{i}'] = imp.fit_transform(globals()[f'dt10_{i}'].copy())
  globals()[f'MSE_LR10_{i}'] = mean_squared_error(data1,globals()[f'imputed_LR_{i}'])
t_end = time.time()
t_LR10 = (t_end- t_start)

[IterativeImputer] Completing matrix with shape (153, 26)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.05
[IterativeImputer] Change: 117.48931746379694, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 2/10, elapsed time 0.07
[IterativeImputer] Change: 30.364167466701417, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 3/10, elapsed time 0.10
[IterativeImputer] Change: 9.474084892910128, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 4/10, elapsed time 0.13
[IterativeImputer] Change: 7.739758800375043, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 5/10, elapsed time 0.16
[IterativeImputer] Change: 4.218981531824966, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 6/10, elapsed time 0.19
[IterativeImputer] Change: 2.7713687054140337, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 7/10, elapsed time 0.21
[IterativeImputer] Change: 2.198100255895724

/usr/local/lib/python3.8/dist-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


[IterativeImputer] Ending imputation round 4/10, elapsed time 0.12
[IterativeImputer] Change: 29.891936678545264, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 5/10, elapsed time 0.14
[IterativeImputer] Change: 13.587280052516888, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 6/10, elapsed time 0.17
[IterativeImputer] Change: 7.154342812008409, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 7/10, elapsed time 0.20
[IterativeImputer] Change: 4.735301943898548, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 8/10, elapsed time 0.22
[IterativeImputer] Change: 3.3640143452969298, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 9/10, elapsed time 0.25
[IterativeImputer] Change: 2.424512052228863, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 10/10, elapsed time 0.28
[IterativeImputer] Change: 1.7579640565347496, scaled tolerance: 0.274 
[IterativeImputer] Completing

/usr/local/lib/python3.8/dist-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


[IterativeImputer] Ending imputation round 2/10, elapsed time 0.06
[IterativeImputer] Change: 15.513541451724448, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 3/10, elapsed time 0.08
[IterativeImputer] Change: 11.976456808728528, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 4/10, elapsed time 0.11
[IterativeImputer] Change: 4.177584604590891, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 5/10, elapsed time 0.14
[IterativeImputer] Change: 2.6570685539392116, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 6/10, elapsed time 0.16
[IterativeImputer] Change: 2.145746243424366, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 7/10, elapsed time 0.19
[IterativeImputer] Change: 1.6089766756656907, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 8/10, elapsed time 0.22
[IterativeImputer] Change: 1.1389122575070658, scaled tolerance: 0.274 
[IterativeImputer] Ending imp

...

/usr/local/lib/python3.8/dist-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


[IterativeImputer] Ending imputation round 4/10, elapsed time 0.11
[IterativeImputer] Change: 14.075171883431025, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 5/10, elapsed time 0.14
[IterativeImputer] Change: 8.496175558137452, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 6/10, elapsed time 0.16
[IterativeImputer] Change: 8.96475524048077, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 7/10, elapsed time 0.19
[IterativeImputer] Change: 6.1774225614784655, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 8/10, elapsed time 0.22
[IterativeImputer] Change: 3.525265322428936, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 9/10, elapsed time 0.25
[IterativeImputer] Change: 1.9295382753855392, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 10/10, elapsed time 0.28
[IterativeImputer] Change: 1.3544942253738554, scaled tolerance: 0.274 
[IterativeImputer] Completing 

/usr/local/lib/python3.8/dist-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


[IterativeImputer] Ending imputation round 1/10, elapsed time 0.03
[IterativeImputer] Change: 64.00702346804128, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 2/10, elapsed time 0.06
[IterativeImputer] Change: 12.023621153238052, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 3/10, elapsed time 0.09
[IterativeImputer] Change: 6.686932752568298, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 4/10, elapsed time 0.12
[IterativeImputer] Change: 3.494348475096572, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 5/10, elapsed time 0.14
[IterativeImputer] Change: 2.020771874689072, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 6/10, elapsed time 0.17
[IterativeImputer] Change: 1.2626813250581677, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 7/10, elapsed time 0.20
[IterativeImputer] Change: 0.9933426466380979, scaled tolerance: 0.274 
[IterativeImputer] Ending imput

/usr/local/lib/python3.8/dist-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [ ]:
mse_LR10_total = 0 
for i in range(1,1001):
  mse_LR10_total = mse_LR10_total + globals()[f'MSE_LR10_{i}']
AMSE_LR10 = mse_LR10_total/1000
print(f'AMSE_LR = {AMSE_LR10}')

AMSE_LR = 5.833825328222278


#### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_SVM_{i}'] = svm_svr.fit_transform(globals()[f'dt10_{i}'].copy())
  globals()[f'MSE_SVM10_{i}'] = mean_squared_error(data1,globals()[f'imputed_SVM_{i}'])
t_end = time.time()
t_SVM10 = (t_end- t_start)

In [ ]:
mse_SVM10_total = 0 
for i in range(1,1001):
  mse_SVM10_total = mse_SVM10_total + globals()[f'MSE_SVM10_{i}']
AMSE_SVM10 = mse_SVM10_total/1000
print(f'AMSE_SVM = {AMSE_SVM10}')

AMSE_SVM = 12.024528836464798


### Imputation using the EM

In [ ]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_EM_{i}'] = impy.em(globals()[f'dt10_{i}'].values.copy())
  globals()[f'MSE_EM10_{i}'] = mean_squared_error(data1,globals()[f'imputed_EM_{i}'])
t_end = time.time()
t_EM10 = (t_end- t_start)

In [ ]:
mse_EM10_total = 0 
for i in range(1,1001):
  mse_EM10_total = mse_EM10_total + globals()[f'MSE_EM10_{i}']
AMSE_EM10 = mse_EM10_total/1000
print(f'AMSE_EM = {AMSE_EM10}')

AMSE_EM = 22.00584992778104


# 30 %

## แทนค่า NaN

In [ ]:
for i in range(1,1001):
  np.random.seed(i)
  globals()[f'random_column_{i}'] = np.random.choice(list1,int((153*17)*(30/100)),p=list_mis)
  globals()[f'random_column_{i}'] = list(globals()[f'random_column_{i}'])


In [ ]:
for i in range(1,1001):
  for j in globals()[f'random_column_{i}']:
    globals()[f'count_{i}'] = []
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('Weight'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('Height'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('BMI'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('Systolic'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('Diastolic'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('HeartRate'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('SMM'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('FatMass'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('Bodyfat'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('waisttohip'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('abdominal'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('visceral'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('001_Glucose'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('UricAcid'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('012_ALT'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('Alkaline_Phos'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('071_HbA1c'))

In [ ]:
count_1

[6, 3, 6, 7, 8, 18, 96, 99, 92, 100, 104, 111, 0, 41, 0, 10, 79]

In [ ]:
list2 = []
list3 = []

for i in range(153):
  m = 1/153
  list2.append(i)
  list3.append(m)

In [ ]:
# random row ไม่ซ้ำกัน
for i in range(1,1001):
  np.random.seed(i)
  globals()[f'dt30_{i}'] = data1.copy()
  for j,k in zip(list1,globals()[f'count_{i}']):
    globals()[f'dt30_{i}'][j].ravel()[np.random.choice(list2,k,p= list3, replace=False)] = np.nan

In [ ]:
dt30_1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,NaN,153.0,20.2,114.0,75.0,69.0,NaN,NaN,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,NaN,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,NaN
2,2,62.0,54.4,165.0,20.0,95.0,75.0,NaN,NaN,NaN,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,NaN
3,2,35.0,56.3,161.0,21.7,95.0,57.0,NaN,NaN,NaN,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,NaN
4,1,46.0,72.3,170.0,NaN,133.0,77.0,107.0,NaN,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,NaN,74.0,69.0,NaN,NaN,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,NaN,NaN,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,NaN
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,NaN,...,0.8,NaN,212.0,68.0,63.0,135.0,11.0,16.0,33.0,NaN
151,2,61.0,61.8,159.0,24.4,117.0,60.0,NaN,NaN,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,NaN,NaN


## Imputation

### Imputation using Mean

In [44]:
from sklearn.impute import SimpleImputer

In [45]:
t_start = time.time()
mean_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean_{i}'] = mean_imputer.fit_transform(globals()[f'dt30_{i}'].copy())
  globals()[f'MSE_MEAN30_{i}'] = mean_squared_error(data1,globals()[f'imputed_mean_{i}'])
t_end = time.time()
t_MEAN10 = (t_end- t_start)

In [46]:
mse_MEAN30_total = 0 
for i in range(1,1001):
  mse_MEAN30_total = mse_MEAN30_total + globals()[f'MSE_MEAN30_{i}']
AMSE_mean30 = mse_MEAN30_total/1000
print(f'AMSE_mean = {AMSE_mean30}')

AMSE_mean = 34.99337102496421


### Imputation using the Miss Forest

In [ ]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF = MissForest()
for i in range(1,1001):
  globals()[f'imputed_MF_{i}'] = MF.fit_transform(globals()[f'dt30_{i}'].copy())
  globals()[f'MSE_MF30_{i}'] = mean_squared_error(data1,globals()[f'imputed_MF_{i}'])
t_end = time.time()
t_MF30 = (t_end- t_start)

In [53]:
mse_MF30_total = 0 
for i in range(1,1001):
  mse_MF30_total = mse_MF30_total + globals()[f'MSE_MF30_{i}']
AMSE_MF30 = mse_MF30_total/1000
print(f'AMSE_MF = {AMSE_MF30}')

AMSE_MF = 13.441194503745011


### Imputation using the KNN

In [47]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [48]:
t_start = time.time()
KNN = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN_{i}'] = KNN.fit_transform(globals()[f'dt30_{i}'].copy())
  globals()[f'MSE_KNN30_{i}'] = mean_squared_error(data1,globals()[f'imputed_KNN_{i}'])
t_end = time.time()
t_KNN30 = (t_end- t_start)

In [49]:
mse_KNN30_total = 0 
for i in range(1,1001):
  mse_KNN30_total = mse_KNN30_total + globals()[f'MSE_KNN30_{i}']
AMSE_KNN30 = mse_KNN30_total/1000
print(f'AMSE_KNN = {AMSE_KNN30}')

AMSE_KNN = 29.284278586468147


### Imputation using the MICE

In [54]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

#### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_LR_{i}'] = imp.fit_transform(globals()[f'dt30_{i}'].copy())
  globals()[f'MSE_LR30_{i}'] = mean_squared_error(data1,globals()[f'imputed_LR_{i}'])
t_end = time.time()
t_LR10 = (t_end- t_start)

In [64]:
mse_LR30_total = 0 
for i in range(1,1001):
  mse_LR30_total = mse_LR30_total + globals()[f'MSE_LR30_{i}']
AMSE_LR30 = mse_LR30_total/1000
print(f'AMSE_LR = {AMSE_LR30}')

AMSE_LR = 2403.3520636062262


#### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_SVM_{i}'] = svm_svr.fit_transform(globals()[f'dt30_{i}'].copy())
  globals()[f'MSE_SVM30_{i}'] = mean_squared_error(data1,globals()[f'imputed_SVM_{i}'])
t_end = time.time()
t_SVM30 = (t_end- t_start)

In [58]:
mse_SVM30_total = 0 
for i in range(1,1001):
  mse_SVM30_total = mse_SVM30_total + globals()[f'MSE_SVM30_{i}']
AMSE_SVM30 = mse_SVM30_total/1000
print(f'AMSE_SVM = {AMSE_SVM30}')

AMSE_SVM = 38.004636804475346


### Imputation using the EM

In [50]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_EM_{i}'] = impy.em(globals()[f'dt30_{i}'].values.copy())
  globals()[f'MSE_EM30_{i}'] = mean_squared_error(data1,globals()[f'imputed_EM_{i}'])
t_end = time.time()
t_EM30 = (t_end- t_start)

In [60]:
mse_EM30_total = 0 
for i in range(1,1001):
  mse_EM30_total = mse_EM30_total + globals()[f'MSE_EM30_{i}']
AMSE_EM30 = mse_EM30_total/1000
print(f'AMSE_EM = {AMSE_EM30}')

AMSE_EM = 72.53501302228244


# AMSE

In [ ]:
amse_value = {'Method':['MEAN','Miss Forest','KNN','MICE-LR','MICE-SVM','EM'],'10%':[AMSE_mean10 ,AMSE_MF10 ,AMSE_KNN10 ,AMSE_LR10 ,AMSE_SVM10 ,AMSE_EM10 ]}

In [ ]:
data_AMSE10 = pd.DataFrame.from_dict(amse_value)
data_AMSE10

,Method,10%
0,MEAN,11.462441
1,Miss Forest,2.165797
2,KNN,8.131331
3,MICE-LR,5.833825
4,MICE-SVM,12.024529
5,EM,22.005850


30%

In [61]:
amse30_value = {'Method':['MEAN','Miss Forest','KNN','MICE-LR','MICE-SVM','EM'],'30%':[AMSE_mean30 ,AMSE_MF30 ,AMSE_KNN30 ,AMSE_LR30 ,AMSE_SVM30 ,AMSE_EM30 ]}

In [65]:
data_AMSE30 = pd.DataFrame.from_dict(amse30_value)
data_AMSE30

,Method,30%
0,MEAN,34.993371
1,Miss Forest,13.441195
2,KNN,29.284279
3,MICE-LR,2403.352064
4,MICE-SVM,38.004637
5,EM,72.535013
